#First we start with the raw data which in this case are raw 1920 * 1080 pixels videos
##Preprocessing Steps:
* First we slice the videos into images. 
* The images are taken at one image per 8 frames.
* We then crop the images near to the faces only to reduce the dimensions. 
* Then we'll pass the images through a clustering algorithm. (KMeans in this case)
* After that we'll be moving the clustered image into its respective folder manually.
* Uptil here we should have a dataset with noise and labelled images ready to train a model.(A seperate folder labelled noise will be used to improve the networks learning)

In [ ]:
import cv2
import math
import os
imagesFolder = "/content/drive/MyDrive/Mirzapur_img_classification/Mirza_img/"
source_file = '/content/drive/MyDrive/Mirzapur_img_classification/raw_data/Mirzapur.S02.HQ.1080p.AMZN.WEBRip.x264.DDP.5.1 - xRG/'
files_source = os.listdir(source_file)
files_dest = os.listdir(imagesFolder)
for i, j in zip(files_source, files_dest):
  videoFile = os.path.join(source_file, i)
  destFile = os.path.join(imagesFolder, j)
  destFile = destFile + "/"
  cap = cv2.VideoCapture(videoFile)
  frameRate = cap.get(5) #frame rate
  while(cap.isOpened()):
      frameId = cap.get(1) #current frame number
      ret, frame = cap.read()
      if (ret != True):
          break
      if (frameId % math.floor(frameRate) == 0):
          filename = destFile + "/image_" +  str(int(frameId)) + ".jpg"
          cv2.imwrite(filename, frame)
  cap.release()
  print('{} file done and saved in {}'.format(i, j))


Mirzapur.S02E01.DHENKUL.1080p.AMZN.WEBRip.x264.DDP.5.1 - xRG.mkv file done and saved in img_6
Mirzapur.S02E02.KHARGOSH.1080p.AMZN.WEBRip.x264.DDP.5.1 - xRG.mkv file done and saved in img_3
Mirzapur.S02E03.VIKLAANG.QUOTA.1080p.AMZN.WEBRip.x264.DDP.5.1 - xRG.mkv file done and saved in img_2
Mirzapur.S02E04.BHAYMUKT..AMZN.WEBRip.x264.DDP.5.1 - xRG.mkv file done and saved in img_1
Mirzapur.S02E05.LANGDA.1080p.AMZN.WEBRip.x264.DDP.5.1 - xRG.mkv file done and saved in img_7
Mirzapur.S02E06.ANKUSH.1080p.AMZN.WEBRip.x264.DDP.5.1 - xRG.mkv file done and saved in img_5
Mirzapur.S02E07.OOD.BILAAV.1080p.AMZN.WEBRip.x264.DDP.5.1 - xRG.mkv file done and saved in img_4 (1)


In [ ]:
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
import shutil
import glob 
# from simple_colors import red, green, cyan
import progressbar
from time import sleep
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    pixels = pyplot.imread(filename)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    if len(results) == 0:
        return None
    else:
        x1, y1, width, height = results[0]['box']
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = asarray(image)
        return face_array

source_path = "G://Code//Machine learning//Projects//Mirza_classification//raw_imgs//imgs//Mirza_img_raw_s02//*"
dest_uncrop_path = 'G://Code//Machine learning//Projects//Mirza_classification//raw_imgs//imgs//uncropped_imgs//'
dest_crop_path = 'G://Code//Machine learning//Projects//Mirza_classification//raw_imgs//imgs//cropped_imgs//'
files = glob.glob(source_path)
total = len(files)
bar = progressbar.ProgressBar(maxval = total, widgets = [progressbar.Bar('>', '[', ']'), ' ', 
                                                      progressbar.Percentage(), ' ', progressbar.ETA()])
bar.start()
i = 0
for img in files:
    pixels = extract_face(img)
    i += 1
    if pixels is None:
        # print(red('Face not detected', 'bold'))
        shutil.move(img, dest_uncrop_path + '{}.jpg'.format(i))
        # print('uncropped {}'.format(img))
    else:
        # print(green('Face detected', 'bold'))
        pixels = Image.fromarray(pixels)
        pixels.save(dest_crop_path + '{}.jpg'.format(i))
    bar.update(i)
    sleep(0.1)
    # print(cyan((100 * i) // total + '%', ['bold', 'underlined']))
bar.finish()

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.cluster import KMeans
import os, shutil, glob, os.path
from PIL import Image as pil_image
image.LOAD_TRUNCATED_IMAGES = True 
model = VGG16(weights='imagenet', include_top=False)

# # Variables
# imdir = 'C:/indir/'
# targetdir = "C:/outdir/"
number_clusters = 17

# Loop over files and get features
filelist = glob.glob(os.path.join(imdir, '*.jpg'))
filelist.sort()
featurelist = []
for i, imagepath in enumerate(filelist):
    print("    Status: %s / %s" %(i, len(filelist)), end="\r")
    img = image.load_img(imagepath, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = np.array(model.predict(img_data))
    featurelist.append(features.flatten())

# Clustering
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(np.array(featurelist))

# Copy images renamed by cluster 
# Check if target dir exists
try:
    os.makedirs(targetdir)
except OSError:
    pass
# Copy with cluster name
print("\n")
for i, m in enumerate(kmeans.labels_):
    print("    Copy: %s / %s" %(i, len(kmeans.labels_)), end="\r")
    shutil.copy(filelist[i], targetdir + str(m) + "_" + str(i) + ".jpg")